# Validator info collection script

In [1]:
import pygsheets
import imaplib, email
from email.header import decode_header
import re
import pandas as pd

#setting up credentials
ORG_EMAIL   = "@myemail.address"
FROM_EMAIL  = "firstnam.lastname" + ORG_EMAIL
FROM_PWD    = "mypw"
SMTP_SERVER = "my server"
SMTP_PORT   = 993

list_Subject = []
list_From = []
list_Date = []
list_PubKey =[]

mail = imaplib.IMAP4_SSL(SMTP_SERVER)
mail.login(FROM_EMAIL,FROM_PWD)

#define range: a specific group within "Chain"
mail.select('Chain')

#information gathering
for num in mail.search(None,'TO','myemail address')[1][0].split():
    status, msg = mail.fetch(num, '(RFC822)')
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject = decode_header(msg["Subject"])[0][0]
            from_ = decode_header(msg["From"])[0][0]
            date_ = decode_header(msg["Date"])[0][0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode()
            # email sender
            #from_ = msg.get("From")
            #date_ = msg.get("Date")
            #print("Subject:", subject)
            
            list_Subject.append(subject)
            list_From.append(from_)
            list_Date.append(date_)
            
            print("From:", from_)
            print("Date:", date_)
        
            #print("Date2:", date)
        else:
            print('NA3')
            list_PubKey.append('NA3')
            
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                        words = body.split(" ")
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        #refine my search results
                        for word in words:
                            if re.match(r'^certikvalconspub1[a-zA-Z0-9]{47}',word):
                                word2 = word.rstrip().replace('>', '').replace(' ', '').replace('\n', '').replace('\r', '')
                                print("The Validatior Pubkey is:" + word.rstrip().replace('>', '').replace(' ', '').replace('\n', '').replace('\r', ''),len(word2))
                                del list_PubKey [-1]
                                print ('NA3 removed with PubKey info')
                                list_PubKey.append(word2) 

                                break
                        else:
                            if 'NA3' in list_PubKey:
                                del list_PubKey[-1]
                                print ('NA3 replaced with NA')
                                list_PubKey.append('NA')
                            else:
                                list_PubKey.append('NA')
                                

            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                words = body.split(" ")
                if content_type == "text/plain":
                    # print only text email parts
                    #print(body)
                    if 'NA3' in list_PubKey:
                        del list_PubKey[-1]
                        print ('NA3 replaced with NA2')
                        list_PubKey.append('NA2')
                    else:    
                        print ('NA2')
                        list_PubKey.append('NA2')
                    for word in words:
                        if re.match(r'^certikvalconspub1[a-zA-Z0-9]{47}',word):
                            print(" The Validatior Pubkey is: " + word.rtrip().replace('>', '').replace(' ', '').replace('\n', '').replace('\r', ''), len(word2))
                            word2 = word.rstrip().replace('>', '').replace(' ', '').replace('\n', '').replace('\r', '')
                            del list_PubKey [-1]
                            print ('NA 3 removed with PubKey info')
                            list_PubKey.append(word2)
                            break

         

            print("="*100)
            
#print(list_Subject,len(list_Subject))
#print("-"*100)
#print(list_From, len(list_From))
#print("-"*100)
#print(list_Date, len(list_Date))
#print("-"*100)
#print(list_PubKey, len(list_PubKey))
#print (len(list_Date))

From: mihbergen@gmail.com
Date: Sat, 18 Jul 2020 02:35:49 +0300
NA3
NA3 replaced with NA
From: b'L\xc3\xaa Thanh'
Date: Sat, 18 Jul 2020 06:18:34 +0700
NA3
The Validatior Pubkey is:certikvalconspub1zcjduepqnkg5dpc7v0e397s23ughkuxympawhfkp2ccjhdzugmc499tf9skqzwzawv 83
NA3 removed with PubKey info
From: CertiK Chain <chain@certik.org>
Date: Thu, 16 Jul 2020 22:09:29 -0400
NA3
The Validatior Pubkey is:certikvalconspub1zcjduepqnkg5dpc7v0e397s23ughkuxympawhfkp2ccjhdzugmc499tf9skqzwzawv 83
NA3 removed with PubKey info
From: b'L\xc3\xaa Thanh'
Date: Fri, 17 Jul 2020 07:45:18 +0700
NA3
NA3 replaced with NA
From: CertiK Chain <chain@certik.org>
Date: Fri, 17 Jul 2020 18:43:49 -0400
NA3
The Validatior Pubkey is:certikvalconspub1zcjduepq9ru3w3gn5ra3s9t7ct739c4twpcme5wzsl5csnmrra3uyqr3v8qsrcqjhg 83
NA3 removed with PubKey info
From: "Yandex Mail" <ooss1@yandex.ru>
Date: Sat, 18 Jul 2020 01:21:55 +0300
NA3
The Validatior Pubkey is:certikvalconspub1zcjduepq9ru3w3gn5ra3s9t7ct739c4twpcme5wzsl5csnmrra3

In [2]:
#check lengths of the list
print(len(list_Subject))
print(len(list_From))
print(len(list_Date))
print(len(list_PubKey))


529
529
529
529


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#create dataframe
validator_dict = {'list_Date':list_Date, 'list_From':list_From, 'list_PubKey': list_PubKey}
vd = pd.DataFrame(validator_dict) 
print(vd)

                                  list_Date  \
0           Sat, 18 Jul 2020 02:35:49 +0300   
1           Sat, 18 Jul 2020 06:18:34 +0700   
2           Thu, 16 Jul 2020 22:09:29 -0400   
3           Fri, 17 Jul 2020 07:45:18 +0700   
4           Fri, 17 Jul 2020 18:43:49 -0400   
5           Sat, 18 Jul 2020 01:21:55 +0300   
6           Fri, 17 Jul 2020 18:04:16 -0400   
7           Sat, 18 Jul 2020 00:43:30 +0300   
8           Fri, 17 Jul 2020 18:02:57 -0400   
9           Fri, 17 Jul 2020 23:31:55 +0300   
10          Fri, 17 Jul 2020 18:02:11 -0400   
11          Fri, 17 Jul 2020 20:28:34 +0000   
12          Fri, 17 Jul 2020 18:00:49 -0400   
13          Fri, 17 Jul 2020 23:06:19 +0300   
14          Fri, 17 Jul 2020 17:57:59 -0400   
15          Fri, 17 Jul 2020 22:35:27 +0300   
16          Fri, 17 Jul 2020 14:54:41 -0400   
17          Fri, 17 Jul 2020 14:53:32 -0400   
18          Fri, 17 Jul 2020 17:38:58 +0000   
19          Fri, 17 Jul 2020 17:27:40 +0000   
20          F

In [5]:
vdif = pygsheets.authorize(service_file = 'myfile.json')
vdif = vdif.open('target googlesheet')
wks = vdif[0]
wks.set_dataframe(vd,(2,1))

#end